In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from datetime import date, timedelta
from dotenv import load_dotenv
from selenium.webdriver.common.keys import Keys
import os
import time

# carregando as variáveis de ambiente
load_dotenv()


True

In [2]:
def retornar_periodo():
    hoje = date.today()
    amanha = hoje + timedelta(days=1)
    hoje_str = date.strftime(hoje, '%d/%m/%Y')
    amanha_str = date.strftime(amanha, '%d/%m/%Y')
    periodo_str = (f'{hoje_str} - {amanha_str}')
    return periodo_str


In [3]:
def obter_driver():
    try:
        # obtendo o usuário logado
        usuario = os.getlogin()
        chrome_options = Options()
        chrome_options.add_argument(
            f'user-data-dir=C:/Users/{usuario}/AppData/Local/Google/Chrome/Selenium'
        )
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        # maximixando a tela
        driver.maximize_window()
        return driver
    except Exception as e:
        print(f'Erro ao obter o driver: {e}')
        return False

In [4]:
def logar(driver):
    try:
        # indo para a tela de login
        driver.get("https://qualylab.gerencialab.com.br/")

        # aguardando elemento aparecer na tela
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//img[@src="/assets/images/Logo_Gerencialab-azul.png"]'))
            )
        time.sleep(4)

        # obtendo as variáveis de ambiente
        login = os.getenv('LOGIN')
        password = os.getenv('PASSWORD')

        # preenchendo o campo usuário
        driver.find_element(By.XPATH, '//*[@id="loginsite"]').send_keys(login)

        # preenchendo o campo senha
        driver.find_element(By.XPATH, '//*[@id="senhasite"]').send_keys(password)

        # clicando em acessar
        driver.find_element(By.XPATH, '//*[@id="authLogin"]').click()

        # aguardando logo GerenciaLab Aparecer na tela
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//img[@src="/assets/images/gerencialab-logo-n.png"]'))
            )
        time.sleep(4)

        return True
    except Exception as e:
        print(f'Erro ao logar no sistema: {e}')
        return False

In [5]:
# função que obtém a lista das amostras
def obter_dados(driver):
    try:
        # indo para ordens de serviço
        driver.get("https://qualylab.gerencialab.com.br/service-order")
        # aguardando label Ordem serviços aparecer na tela
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//h4[text()="Ordem serviços"]')))
        time.sleep(4)
        # clicando no botão Limpar Pesquisa
        driver.find_element(By.XPATH, '//span[text()="Limpar Pesquisa"]').click()
        time.sleep(4)
        # obtendo o campo da data de entrega
        campo_data = driver.find_element(By.XPATH, '//*[@id="dataPrazoEntregaOSForm"]')
        campo_data.send_keys(retornar_periodo())

        # clicando em aplicar
        driver.find_element(By.XPATH, '//div[@class="drp-buttons"]//button[text()="Aplicar" and not(@disabled)]').click()
        time.sleep(4)

        # obtendo a lista de elementos
        lista_elementos = driver.find_elements(By.CSS_SELECTOR, '#tableOrdemdeServico tr.even, #tableOrdemdeServico tr.odd')

        amostras = []
        for linha in lista_elementos:
            cliente = linha.find_elements(By.TAG_NAME, 'td')[4].text
            status_os = linha.find_elements(By.TAG_NAME, 'td')[1].text
            amostra = linha.find_elements(By.TAG_NAME, 'td')[2].text
            prioridade = linha.find_elements(By.TAG_NAME, 'td')[3].text
            amostras.append((status_os, amostra, cliente, prioridade))

        return amostras
    except Exception as e:
        print(f'Erro ao obter os dados: {e}')
        return None

In [6]:
def sair_sistema(driver):
    try:
        driver.find_element(By.XPATH, '//a[@href="/sair"]').click()
        time.sleep(4)
        driver.close()
        return True
    except Exception as e:
        print(f'Erro ao fechar o navegador: {e}')
        return False

In [ ]:
# Iniciar automação
def iniciar_automacao():
    driver = obter_driver()
    if not driver:
        return 'Erro ao gerar os'

In [7]:
# testes
driver = obter_driver()
if not driver:
    exit()
logado = logar(driver)
if not logado:
    exit()

dados = obter_dados(driver)
print(dados)

saiu = sair_sistema(driver)
if not saiu:
    exit()

[('00000030/2025', 'OS00066/2025', '00000030804', 'Aguardando agendamento'), ('00000023/2025', 'OS00051/2025', '00000030787', 'Aguardando agendamento'), ('00000023/2025', 'OS00050/2025', '00000030785', 'Aguardando agendamento'), ('00000023/2025', 'OS00050/2025', '00000030786', 'Aguardando agendamento'), ('00000025/2025', 'OS00046/2025', '00000030779', 'Aguardando agendamento'), ('00000025/2025', 'OS00046/2025', '00000030780', 'Aguardando agendamento')]
